### Imports

In [ ]:
import numpy as np
import os
import json 
import glob

### Data Parsing

In [17]:
def printConvo(userData, systemData):
    print("session id:", userData["session-id"])

    print(userData["task-information"]["goal"]["text"])

    temp = 0
    for speach in systemData["turns"]:
        print("system:", speach["output"]["transcript"])
        print("user:", userData["turns"][temp]["transcription"])
        temp += 1

def writeFiles(userData, systemData, conversationsFile, utterancesFile):
    
    conversationsFile.write("session id: %s\n" % userData["session-id"])
    conversationsFile.write("%s\n" % userData["task-information"]["goal"]["text"])

    temp = 0
    for speach in systemData["turns"]:
        
        dialogAct = userData["turns"][temp]["semantics"]["cam"]
        utteranceContent = userData["turns"][temp]["transcription"]
        
        conversationsFile.write("system: %s\n" %  speach["output"]["transcript"])
        conversationsFile.write("user: %s\n" % utteranceContent)
        utterancesFile.write("%s %s\n" % (dialogAct.split("(")[0], utteranceContent.lower()))
        
        temp += 1
    
    conversationsFile.write("\n\n")

def writeFile(waiting):
    if waiting is False:
        conversationsFile = open("convo.txt", "w+")
        utterancesFile = open("utterances.txt", "w+")
    
    userFiles = glob.glob('**/label.json', recursive=True)
    systemFiles = glob.glob('**/log.json', recursive=True)
    i = 0
    for userFile in userFiles:
        with open(userFile) as f:
            userData = json.load(f)
        with open(systemFiles[i]) as f:
            systemData = json.load(f)
        i += 1
        if waiting is True :
            printConvo(userData, systemData)
            input('press Enter to display another chat both conversation')
        else: 
            writeFiles(userData, systemData, conversationsFile, utterancesFile)

    if waiting is False: 
        conversationsFile.close()
        utterancesFile.close()

writeFile(False)

### Splitting Sets

In [23]:
def createDataSets():
    utterancesFile = open("utterances.txt", "r")
    utterancesData = {}
    for line in utterancesFile:
        line = line.split(" ", 1)
        if line[0] not in utterancesData:
            utterancesData[line[0]] = []    
        utterancesData[line[0]].append(line[1].strip())

    trainingData = {}
    testData = {}
    for dialogType in utterancesData:
        splitted = np.split(utterancesData[dialogType], [int(len(utterancesData[dialogType]) * 0.85)])
        trainingData[dialogType] = splitted[0].tolist()
        testData[dialogType] = splitted[1].tolist()
        
    return trainingData, testData

createDataSets()

({'null': ['music',
   'type',
   'noise',
   'noise',
   'noise',
   'what',
   'noise',
   'noise',
   'noise',
   'noise',
   'noise',
   'noise',
   'noise',
   'noise',
   'freakin',
   'unintelligible',
   'noise',
   'inaudible',
   'noise',
   'uh',
   'medium',
   'sil',
   'inaudible',
   'noise',
   'sil',
   'what is the',
   'okay and um',
   'range please',
   'sil',
   'unintelligible',
   'noise',
   'sil',
   'unintelligible',
   'sil',
   'noise',
   'noise',
   'noise',
   'noise',
   'sil',
   'unintelligible',
   'unintelligible',
   'unintelligible',
   'unintelligible',
   'unintelligible',
   'unintelligible',
   'unintelligible',
   'unintelligible',
   'unintelligible',
   'unintelligible',
   'noise',
   'noise',
   'noise',
   'cough',
   'noise',
   'uh',
   'noise',
   'noise',
   'uh unintelligible',
   'sil',
   'okay',
   'sil',
   'unintelligible',
   'um breath',
   'care',
   'unintelligible',
   'i need the nor',
   'sil',
   'sil',
   'sil',
   'si

### Rule Based

In [19]:
def ruleBasedClassifier() 
keywords = {"affirm":["yes", "yeah", "yea"],
            "confirm":["is it", "does it"],
            "bye": ["bye"],
            "deny":["no", "do not", "dont"],
            "hello":["hi", "hello"],
            "inform":["any", "dont care", "do not care", "doesnt matter", "does not matter","south",
            "north","east","west","centre", "center", "expensive", "moderately","moderate","cheap",
            "creative", "christmas","halal", "vegetarian","indian","cantonese","american","persian",
            "european","chinese","sea food","spanish","portuguese","italian","mediterranean","gastropub",
            "steak","bistro","british","japanese","danish","lebanese","caribbean","thai","asian","welsh",
            "french","australian","brazilian","irish","english","polynesian","corsica","vietnamese","turkish",
            "mexican","moroccan"],
            "negate":["no"],
            "repeat":["repeat"],
            "requalts": ["what about","how about","else","anything","different"],
            "reqmore":["more"],
            "request":["address","area","location","type", "type of food","price","phone","phonenumber","telephone","post code", "postcode"],
            "restart":["start over", "restart"],
            "ack" : ["okay"],
            "thankyou":["thank you", "thanks"],
            "null": ["noise", "unintelligible"],}

#opens the list of utterances (will have to be changed to the dictionary)
utterances = open("utterances.txt", "r")
writingFile = open("RuleBasedResult.txt", "w")

#Counters in order to calculate the accuracy
amountRight = 0
total = 0

#Loop through the file
for line in utterances:
    total = total +1

    #Splits the sentence, removes the category
    fullsentence = line
    sentence = fullsentence.split(" ", 1)[1]
    actCategory = fullsentence.split(" ", 1)[0]
   
    #Loops through the categories and checks whether a the keyword is in the sentence
    flag = False
    for category in keywords:
        for keyword in keywords[category]:
            if keyword in sentence:
                string = category + " " + sentence
                writingFile.write(string)

                #Counts for the proportion
                if category == actCategory:
                    amountRight = amountRight +1
                    flag = True
                    break
        if flag is True:
            break
            

proportion = (amountRight/total)*100
print("Accuracy = ", proportion, "%")
utterances.close()
writingFile.close()

Total =  25501
Accuracy =  82.9496882475197 %
